<a href="https://colab.research.google.com/github/mahoangnhatphi/prj/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 0: Import Datasets

In [1]:
%%time
# https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip

import requests, zipfile, io, os, shutil

root = '/content'
dogimages_url = "https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip"
humanimages_url = "https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/lfw.zip"

data_path = os.path.join(root,'data')
dogimages_path = os.path.join(data_path,'dogImages')
humanimages_path = os.path.join(data_path,'lfw')
data_path = os.path.join(root,'data')
if not os.path.exists(data_path): os.mkdir(data_path)

# if os.path.exists(dogimages_path):shutil.rmtree(dogimages_path)
# if os.path.exists(humanimages_path):shutil.rmtree(humanimages_path)

if not os.path.exists(dogimages_path):
  print("downloading dog images")
  r = requests.get(dogimages_url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(data_path)

if not os.path.exists(humanimages_path):
  print("downloading human images")
  r = requests.get(humanimages_url)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall(data_path)

import numpy as np
from glob import glob

# load filenames for human and dog images
human_files = np.array(glob("/content/data/lfw/*/*"))
dog_files = np.array(glob("/content/data/dogImages/*/*/*"))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

downloading dog images
downloading human images
There are 13233 total human images.
There are 8351 total dog images.
CPU times: user 10.3 s, sys: 4.56 s, total: 14.9 s
Wall time: 38.5 s


In [2]:
if not os.path.exists('/content/haarcascades'): os.mkdir('/content/haarcascades')
haarcascades_url = 'https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml'
!wget -O /content/haarcascades/haarcascade_frontalface_alt.xml {haarcascades_url}

--2023-09-01 11:17:13--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 676709 (661K) [text/plain]
Saving to: ‘/content/haarcascades/haarcascade_frontalface_alt.xml’

/content/haarcascad 100%[===================>] 660.85K  --.-KB/s    in 0.04s   

2023-09-01 11:17:13 (14.4 MB/s) - ‘/content/haarcascades/haarcascade_frontalface_alt.xml’ saved [676709/676709]



Step 1: Detect Dogs

In [3]:
# Write a Dog Detector

In [8]:
#Obtain Pre-trained VGG-16 Model
import torch
import torchvision.models as models

# define VGG16 model
VGG16 = models.vgg16(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()

# move model to GPU if CUDA is available
if use_cuda:
    VGG16 = VGG16.cuda()

In [9]:
from PIL import Image
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets
import os

def VGG16_predict(img_path):
    '''
    Use pre-trained VGG-16 model to obtain index corresponding to
    predicted ImageNet class for image at specified path

    Args:
        img_path: path to an image

    Returns:
        Index corresponding to VGG-16 model's prediction
    '''

    ## TODO: Complete the function.
    ## Load and pre-process an image from the given img_path
    ## Return the *index* of the predicted class for that image

    # data transformation
    batch_size = 64
    img = Image.open(img_path).convert('RGB')
    image_transforms = transforms.Compose([
                            transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                            transforms.RandomRotation(degrees=15),
                            transforms.ColorJitter(),
                            transforms.RandomHorizontalFlip(),
                            transforms.CenterCrop(size=224),  # Image net standards
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406],
                                                 [0.229, 0.224, 0.225])  # Imagenet standards
                        ])

    image = image_transforms(img)[:3,:,:].unsqueeze(0)
#     image = image_transforms(img)
#     print(image_transformation)

    if use_cuda:
        image = image.cuda()
    output = VGG16(image)

    _,pred = torch.max(output, dim=1)
    pred=pred.cpu()
    pred = pred.data.numpy()[0]

    return pred # predicted class index

VGG16_predict(dog_files[0])

189

In [11]:
### returns "True" if a dog is detected in the image stored at img_path
def dog_detector(img_path):
    ## TODO: Complete the function.
#     in VGG16 index 151 to 268 are dog classifications

    return VGG16_predict(img_path)>= 151 and VGG16_predict(img_path)<=268 # true/false

print(dog_detector(dog_files[7]))

True


Assess the Dog Detector

In [12]:
### TODO: Test the performance of the dog_detector function
### on the images in human_files_short and dog_files_short.

human_files_count=0
dog_files_count=0
for i in range(len(human_files_short)):
    if dog_detector(human_files_short[i]):
        human_files_count+=1
    if dog_detector(dog_files_short[i]):
        dog_files_count+=1
print("number of dog faces detected in human_fiiles_short {0}".format(human_files_count))
print("number of dog faces detected in dog_files_short {0}".format(dog_files_count))


NameError: ignored

In [13]:
# CNN
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_path = '/content/data/dogImages/train/'
val_path = '/content/data/dogImages/valid'
test_path = '/content/data/dogImages/test'

batch_size=64
train_generator = train_datagen.flow_from_directory(
        train_path,
        # '/content/cats_dogs/data/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_path,
        # '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = val_datagen.flow_from_directory(
        test_path,
        # '/content/cats_dogs/data/val',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 6680 images belonging to 133 classes.
Found 835 images belonging to 133 classes.
Found 836 images belonging to 133 classes.


In [14]:
from time import time
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=( 150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(133))
model.add(Activation('softmax'))

In [15]:
# Specify Loss Function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Train and Validate the model

In [ ]:
start_time = time()
model.fit_generator(
        train_generator,
        #steps_per_epoch=18631 // batch_size,
        epochs=16,
        validation_data=validation_generator,
        #validation_steps=10119 // batch_size
        )
model.save_weights('first_try.h5')

print('time taken ',time()-start_time)

<ipython-input-16-919974f827fb>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/16
 19/105 [====>.........................] - ETA: 4:18 - loss: 4.8932 - accuracy: 0.0074

Test the Model

In [ ]:
# test_prediction = model.predict(test_generator)
model.metrics_names

In [ ]:
# from sklearn.metrics import accuracy_score
# model.evaluate_generator(test_generator)
test_generator.reset()
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_eval = model.evaluate_generator(test_generator,STEP_SIZE_TEST)
print('test loss ',test_eval[0])
print('test accuracy ',test_eval[1])